# Feature Engineering

Convert data into more ML friendly formats.  Reversible so the model output later can be reverted back to TLE style format.

This conversion needs to be performed on all datasets.

Features:

| Column        | Desc  | Effect on SGP4 |
| :------------- | :------| :----|
| `NORAD_CAT_ID` | Satellite identifier, not used in training, no action needed |
| `OBJECT_TYPE` | Satellite meta data, not used in training, no action needed (only in `full` version) |
| `TLE_LINE1` | Actual TLE line 1, not used in training, no action needed (only in `full` version) |
| `TLE_LINE2` | Actual TLE line 2, not used in training, no action needed (only in `full` version) |
| `MEAN_MOTION_DOT` | Some sort of scaling may be needed | NOT used in SGP4 propagation |
| `MEAN_MOTION_DDOT` | Some sort of scaling may be needed | NOT used in SGP4 propagation |
| `BSTAR` | Some sort of scaling may be needed | Affects `v` component based on `r` (assuming higher bstar = higher drag = more decay) |
| `INCLINATION` | Convert cyclic 0 .. 180 | Defines path of possible `r` values (0 = equator, 90 = polar orbit) |
| `RA_OF_ASC_NODE` | Convert cyclic 0 .. 360 | Defines path of possible `r` values (kind of like rotating the orbit viewed from the poles?) |
| `ECCENTRICITY` | Some scaling needed, 0 .. 0.25 | Defines path of possible `r` values (0 = circular orbit) |
| `ARG_OF_PERICENTER` | Convert cyclic 0 .. 360 | Defines path of possible `r` values (0 means closest when crossing north-south reference plane) |
| `MEAN_ANOMALY` | Convert cyclic 0 .. 360, this loops multiple times per day and most cycles are unobserved in the data | Defines which `r` position is used |
| `MEAN_MOTION` | > 11.25 | Defines path of possible `r` values (smaller = longer orbit) |
| `REV_AT_EPOCH` | 0-99999, but sometimes inconcsistency in data where there is an offset to this from different ground stations (a guess) | NOT used in SGP4 propagation |
| `EPOCH` | Time, while no scaling is needed, we will need to use this for constructing `X` and `y` | Time and time offset used for propagation |
| `GP_ID` | Unique identifier for the TLE entry, not used in training, no action needed |

While `MEAN_ANOMALY` is represeted in degrees, because a lot of cycles are left out due to how sparse the data is, a combination of `REV_AT_EPOCH` + `MEAN_ANOMALY` may be a better representation of the features rather than using sin/cos representation.  Other conversion can be done without grouping, but due to `REV_AT_EPOCH` rolling over at 100k and inconsistency between ground stations, we might need to handle it per satellite.



Datasets:

```
2_min/train.pkl
0_min/test.pkl
0_min/secret_test.pkl
```

Converting `min` versions only for now to save some memory and disk space.  Can be replaced with `full` if needed.

In [1]:
import pandas as pd
import numpy as np
import os

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt

import sys
sys.path.append('../models/model_0')

from clean_data import __jday_convert

In [2]:
version = "min" # or "min" or "full" data

In [3]:
input_files = [
    (2, "train.pkl"),
    (0, "test.pkl"),
    (0, "secret_test.pkl")
]

for n,f in input_files:
    print(f"{os.environ['GP_HIST_PATH']}/../{n}_{version}/{f}")

train_df = pd.read_pickle(f"{os.environ['GP_HIST_PATH']}/../2_{version}/train.pkl")
# test_df = pd.read_pickle(f"{os.environ['GP_HIST_PATH']}/../0_{version}/test.pkl")
# secret_test_df = pd.read_pickle(f"{os.environ['GP_HIST_PATH']}/../0_{version}/secret_test.pkl")

/mistorage/mads/data/gp_history/../2_min/train.pkl
/mistorage/mads/data/gp_history/../0_min/test.pkl
/mistorage/mads/data/gp_history/../0_min/secret_test.pkl


In [4]:
train_df = pd.read_pickle(f"{os.environ['GP_HIST_PATH']}/../2_{version}/train.pkl")

In [5]:
def convert_feature_values(df):
    name = df.name
    df = df.set_index("EPOCH").sort_index()
    # convert ARG_OF_PERICENTER, RA_OF_ASC_NODE, and MEAN_ANOMALY to non-cyclic version
    df["ARG_OF_PERICENTER_ADJUSTED"] = np.cumsum(np.around(df.ARG_OF_PERICENTER.diff().fillna(0) / -360))*360 + df.ARG_OF_PERICENTER
    df["RA_OF_ASC_NODE_ADJUSTED"] = np.cumsum(np.around(df.RA_OF_ASC_NODE.diff().fillna(0) / -360))*360 + df.RA_OF_ASC_NODE
    
    # this is because for REV_AT_EPOCH = 100,000, it's recorded as 10,000 instead of 0
    # this doesn't handle the case for multiple ground stations reporting though, if the previous is different....
    # would it be better to just remove this as an outlier just to be safe?
    # 90k +- 20 max offset based on MEAN_MOTION maximum from earlier steps
    df.loc[(df.REV_AT_EPOCH==10000) & df.REV_AT_EPOCH.diff().between(-90020,-89980),'REV_AT_EPOCH'] = 0

    # combine REV_AT_EPOCH and MEAN_ANOMALY for a non-cyclic representation
    adjusted_rev = df.REV_AT_EPOCH + np.cumsum(np.around(df.REV_AT_EPOCH.diff().fillna(0) / -100000)) * 100000
    df["REV_MEAN_ANOMALY_COMBINED"] = adjusted_rev * 360 + df.MEAN_ANOMALY
    
    # this is to handle the REV_AT_EPOCH problem inconsistency problem
    # otherwise the REV_MEAN_ANOMALY_COMBINED difference may be incorrect
    # bfill because we may start at non-zero due to previous data removal bit
    a = np.round((adjusted_rev.diff().fillna(method='bfill')/2000))
    df["SUBGROUP"] = np.cumsum(a).astype(int)
    return df

# Leaving here for reference only, not actually used anymore
def revert_feature_values(df):
    df['REV_AT_EPOCH'] = ((df.REV_MEAN_ANOMALY_COMBINED // 360) % 100000).astype(int)
    df['MEAN_ANOMALY'] = df.REV_MEAN_ANOMALY_COMBINED % 360
    df['RA_OF_ASC_NODE'] = df.RA_OF_ASC_NODE_ADJUSTED % 360
    df['ARG_OF_PERICENTER'] = df.ARG_OF_PERICENTER_ADJUSTED % 360
    return df

In [6]:
converted_df = train_df.groupby(by="NORAD_CAT_ID", as_index=False).progress_apply(convert_feature_values).reset_index()

  0%|          | 0/12298 [00:00<?, ?it/s]

In [8]:
def generate_X_y(df):
    idx = df.name
    
    df = df[['NORAD_CAT_ID', 'EPOCH', 'BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY', 'ARG_OF_PERICENTER',
             'MEAN_ANOMALY', 'MEAN_MOTION', 'REV_AT_EPOCH',
             'ARG_OF_PERICENTER_ADJUSTED', 'RA_OF_ASC_NODE_ADJUSTED', 'REV_MEAN_ANOMALY_COMBINED',
             'GP_ID']]
    df = df.drop_duplicates(subset=['EPOCH']).sort_values("EPOCH")
    dfs = []
    for i in range(1,11):
        dfi = pd.concat([df,df.shift(-i).add_suffix("_b")], axis=1).dropna()
        dfs.append(dfi)
    ddf = pd.concat(dfs)

    # Reference variables only, DO NOT USE TO TRAIN
    df = ddf[['NORAD_CAT_ID','GP_ID','GP_ID_b','EPOCH','EPOCH_b']]
    df.columns = ['__NORAD_CAT_ID','__GP_ID_1','__GP_ID_2','__EPOCH_1','__EPOCH_2']
    df['__GP_ID_2'] = df['__GP_ID_2'].astype(int)
    
    # X
    x_cols = ['BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE', 'ECCENTRICITY', 'ARG_OF_PERICENTER',
              'MEAN_ANOMALY', 'MEAN_MOTION', 'REV_AT_EPOCH']
    df[['X_EPOCH_JD', 'X_EPOCH_FR']] = ddf.EPOCH.apply(__jday_convert).to_list()
    df[['X_'+x for x in x_cols]] = ddf[x_cols]
    df['X_delta_EPOCH'] = (ddf.EPOCH_b - ddf.EPOCH).astype(int) / 86400000000000
    # y
    df['y_delta_INCLINATION'] = ddf.INCLINATION_b - ddf.INCLINATION
    df['y_delta_ECCENTRICITY'] = ddf.ECCENTRICITY_b - ddf.ECCENTRICITY
    df['y_delta_MEAN_MOTION'] = ddf.MEAN_MOTION_b - ddf.MEAN_MOTION
    df['y_delta_ARG_OF_PERICENTER'] = ddf.ARG_OF_PERICENTER_ADJUSTED_b - ddf.ARG_OF_PERICENTER_ADJUSTED
    df['y_delta_RA_OF_ASC_NODE'] = ddf.RA_OF_ASC_NODE_ADJUSTED_b - ddf.RA_OF_ASC_NODE_ADJUSTED
    df['y_delta_REV_MEAN_ANOMALY_COMBINED'] = ddf.REV_MEAN_ANOMALY_COMBINED_b - ddf.REV_MEAN_ANOMALY_COMBINED
    
    # not sure if this day limiting thing makes sense....
    df = df[(df['X_delta_EPOCH'] < 5) & (df['X_delta_EPOCH'] > 0.25)]
    return df

In [15]:
# %%time

# sample_df = converted_df[converted_df.INCLINATION.between(65,67)]
# random_ids = np.append(np.random.choice(sample_df.NORAD_CAT_ID.unique(), 16), [10615, 10417, 36954, 21723])
# sample_df = sample_df[sample_df.NORAD_CAT_ID.isin(random_ids)]
# processed_sample_df = sample_df.groupby(["NORAD_CAT_ID","SUBGROUP"], as_index=False).progress_apply(generate_X_y).reset_index(drop=True)

# # save sample
# processed_sample_df.to_pickle(f"{os.environ['GP_HIST_PATH']}/../3_min/sample_train.pkl")

  0%|          | 0/37 [00:00<?, ?it/s]

CPU times: user 9.16 s, sys: 422 ms, total: 9.58 s
Wall time: 9.97 s


In [ ]:
processed_df = converted_df.groupby(["NORAD_CAT_ID","SUBGROUP"], as_index=False).progress_apply(generate_X_y).reset_index(drop=True)

  0%|          | 0/40240 [00:00<?, ?it/s]

In [14]:
processed_df

,__NORAD_CAT_ID,__GP_ID_1,__GP_ID_2,__EPOCH_1,__EPOCH_2,X_EPOCH_JD,X_EPOCH_FR,X_BSTAR,X_INCLINATION,X_RA_OF_ASC_NODE,...,X_MEAN_ANOMALY,X_MEAN_MOTION,X_REV_AT_EPOCH,X_delta_EPOCH,y_delta_INCLINATION,y_delta_ECCENTRICITY,y_delta_MEAN_MOTION,y_delta_ARG_OF_PERICENTER,y_delta_RA_OF_ASC_NODE,y_delta_REV_MEAN_ANOMALY_COMBINED
0,262,44171751,44171752,1990-01-01 11:49:16.590431,1990-01-02 11:40:30.356543,2447892.5,0.492553,0.001613,66.4639,28.4188,...,266.3467,14.087521,45329,0.993909,-0.0006,-0.000008,0.000070,-0.6830,-2.5345,5040.6789
1,262,44171752,44171753,1990-01-02 11:40:30.356543,1990-01-03 11:31:43.824576,2447893.5,0.486462,0.001641,66.4633,25.8843,...,267.0256,14.087591,45343,0.993906,-0.0002,-0.000004,0.000066,-0.6859,-2.5342,5040.6827
2,262,44171753,44171754,1990-01-03 11:31:43.824576,1990-01-04 11:22:56.995392,2447894.5,0.480368,0.001656,66.4631,23.3501,...,267.7083,14.087657,45357,0.993902,0.0000,-0.000002,0.000063,-0.6798,-2.5341,5040.6771
3,262,44171754,17179322,1990-01-04 11:22:56.995392,1990-01-06 11:05:22.554239,2447895.5,0.474271,0.001659,66.4631,20.8160,...,268.3854,14.087720,45371,1.987796,0.0011,0.000001,0.000113,-1.3514,-5.0675,10081.3485
4,262,17179322,44171755,1990-01-06 11:05:22.554239,1990-01-07 22:52:11.029152,2447897.5,0.462067,0.001633,66.4642,15.7485,...,269.7339,14.087833,45399,1.490839,0.0012,0.000004,0.000079,-1.0141,-3.8002,7561.0170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649905,36954,93563559,93628581,2013-08-12 04:31:55.661087,2013-08-17 03:33:44.311391,2456516.5,0.188839,0.000135,65.0086,69.2231,...,94.4638,14.317480,17929,4.959591,0.0003,0.000019,0.000026,9.6007,-13.8943,25551.7596
649906,36954,95351154,95366720,2014-03-07 18:20:23.356607,2014-03-09 17:16:02.328960,2456723.5,0.764159,0.000627,65.0107,207.6514,...,101.0809,14.320507,20901,1.955312,0.0029,0.000106,0.000037,-4.3600,-5.4800,10084.0568
649907,36954,95353937,95386359,2014-03-07 23:21:57.625631,2014-03-11 22:53:58.900415,2456723.5,0.973584,0.000476,65.0069,207.0781,...,87.8676,14.320461,20904,3.980570,0.0062,0.000184,0.000150,-15.9619,-11.1676,20536.0198
649908,36954,95366720,95412011,2014-03-09 17:16:02.328960,2014-03-14 16:15:51.985727,2456725.5,0.719471,0.000542,65.0136,202.1714,...,105.1377,14.320544,20929,4.958214,-0.0022,-0.000109,0.000154,5.7677,-13.8948,25554.1078


In [12]:
train_df

,NORAD_CAT_ID,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,BSTAR,INCLINATION,RA_OF_ASC_NODE,ECCENTRICITY,ARG_OF_PERICENTER,MEAN_ANOMALY,MEAN_MOTION,REV_AT_EPOCH,EPOCH,GP_ID
0,18549,1.801000e-05,0.0,0.002592,62.2415,180.1561,0.070489,265.6761,86.2771,12.852684,58561,2004-04-27 14:18:48.216960,2
1,18727,-2.000000e-08,0.0,0.000100,73.3600,345.6887,0.008815,270.3999,88.6911,12.642166,75486,2004-04-27 15:59:40.727904,3
2,19027,1.280000e-05,0.0,0.001076,83.0239,250.9465,0.008493,184.3222,175.7249,13.856401,95359,2004-04-27 19:45:13.686048,5
3,19128,1.320000e-06,0.0,0.000166,70.9841,207.4830,0.020756,161.3777,199.5075,13.715209,79821,2004-04-27 15:43:11.393472,6
4,19242,2.280000e-06,0.0,0.000739,90.1460,192.1834,0.002746,300.4617,59.3655,12.992417,47996,2004-04-27 03:43:04.015775,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55239834,47040,5.020000e-06,0.0,0.000352,100.3388,129.4893,0.006708,69.8194,291.0167,14.029620,65304,2021-04-21 18:21:15.174144,175915050
55239835,47056,4.883000e-05,0.0,0.000726,74.0327,243.8364,0.003218,190.1928,182.7247,14.748841,48496,2021-04-21 15:15:24.708096,175915052
55239836,47107,6.130000e-06,0.0,0.002987,66.0638,230.2248,0.006675,67.9694,292.8394,12.786370,6566,2021-04-21 16:39:29.953152,175915058
55239837,47199,4.112000e-05,0.0,0.000541,97.8943,161.7037,0.000385,65.8106,294.3508,14.806983,2321,2021-04-21 13:56:56.002272,175915068
